In [42]:
import numpy as np

import cirq
from cirq.contrib import paulistring

In [43]:
import sys
sys.path.append('../Cirq/examples')

import supremacy

In [44]:
def ops_to_matrix(*ops):
    c = cirq.Circuit.from_ops(*ops)
    return c.to_unitary_matrix()
q0 = cirq.NamedQubit('q0')
q1 = cirq.NamedQubit('q1')
q2 = cirq.NamedQubit('q2')
q3 = cirq.NamedQubit('q3')
q4 = cirq.NamedQubit('q4')
q5 = cirq.NamedQubit('q5')

In [104]:
circuit = cirq.Circuit.from_ops(
    cirq.TOFFOLI(q0, q1, q2),
    #cirq.TOFFOLI(q0, q1, q2),
    #cirq.TOFFOLI(q0, q1, q2),
    #cirq.TOFFOLI(q0, q1, q2),
)
#circuit = cirq.testing.nonoptimal_toffoli_circuit(q0, q1, q2)
#cirq.google.optimized_for_xmon(circuit)
#cirq.ExpandComposite().optimize_circuit(circuit)
circuit

q0: ───@───
       │
q1: ───@───
       │
q2: ───X───

In [106]:
cirq.CircuitDag.from_circuit(circuit)==cirq.CircuitDag.from_circuit(circuit)

False

In [39]:
c = paulistring.optimized_circuit(circuit)
d=c.to_text_diagram_drawer()
print(d.render(horizontal_spacing=1))
c

0───────@───────────────────@────────────────────@────────────────────────@─[Z]^0.25───────────────
        │                   │                    │                        │
1─X^0.5─@─X^0.5─@─X^0.5─────@─X^-0.5─@─X^-0.5────@─X^0.5─@─[Z]^0.25─X^0.5─@─X^-0.5───@─[Z]^-0.25─X─
                │                    │                   │                           │
2───────────────@─[X]^-0.25──────────@─[X]^-0.25─────────@─[X]^0.25──────────────────@─[X]^0.25────


0: ───────────@───────────────────────────@────────────────────────────@──────────────────────────────────@───[Z]^0.25───────────────────────
              │                           │                            │                                  │
1: ───X^0.5───@───X^0.5───@───X^0.5───────@───X^-0.5───@───X^-0.5──────@───X^0.5───@───[Z]^0.25───X^0.5───@───X^-0.5─────@───[Z]^-0.25───X───
                          │                            │                           │                                     │
2: ───────────────────────@───[X]^-0.25────────────────@───[X]^-0.25───────────────@───[X]^0.25──────────────────────────@───[X]^0.25────────

In [7]:
cx = paulistring.optimized_circuit(cirq.google.optimized_for_xmon(circuit))
d=cx.to_text_diagram_drawer()
print(d.render(horizontal_spacing=1))

In [8]:
sd = paulistring.pauli_string_dag_from_circuit(
paulistring.convert_and_separate_circuit(cx, leave_cliffords=True)[0])
paulistring.pauli_string_optimize.merge_equal_strings(sd)
sd.to_circuit()

In [9]:
c = cirq.Circuit.from_ops(
    cirq.CZ(q0, q1),
    cirq.X(q1) ** 0.5,
    cirq.CZ(q0, q1),
)
c

q0: ───@───────────@───
       │           │
q1: ───@───X^0.5───@───

In [10]:
cirq.MergeInteractions(allow_partial_czs=False).optimize_circuit(c)
c

q0: ───T─────────X^0.5───S^-1─────Y^-0.5───@───S^-1───Y^0.5───Y^0.5───Z^-0.75───
                                           │
q1: ───X^-0.25───Z───────Y^-0.5────────────@───S^-1───Y^0.5───Z───────X^-0.75───

In [11]:
cc =paulistring.optimized_circuit(c)
cc

q0: ───────────@───Z^-0.5───────────
               │
q1: ───Y^0.5───@───(X^0.5-Z^-0.5)───

In [12]:
ccc = paulistring.clifford_optimized_circuit(cc)
ccc

q0: ───────────@───Z^-0.5───────────
               │
q1: ───Y^0.5───@───(X^0.5-Z^-0.5)───

In [13]:
paulistring.pauli_string_optimized_circuit(ccc, move_cliffords=False)

q0: ───────────@───Z^-0.5───────────
               │
q1: ───Y^0.5───@───(X^0.5-Z^-0.5)───

In [14]:
device = cirq.google.XmonDevice(
            measurement_duration=cirq.google.Bristlecone._measurement_duration,
            exp_w_duration=cirq.google.Bristlecone._exp_w_duration,
            exp_11_duration=cirq.google.Bristlecone._exp_z_duration,
            qubits=cirq.google.known_devices._parse_device("""
ABC
ABC
ABC
""")[0])
print(device)

(0, 0)───(0, 1)───(0, 2)
│        │        │
│        │        │
(1, 0)───(1, 1)───(1, 2)
│        │        │
│        │        │
(2, 0)───(2, 1)───(2, 2)


In [15]:
q00 = cirq.GridQubit(0, 0)
q01 = cirq.GridQubit(0, 1)
q02 = cirq.GridQubit(0, 2)
q10 = cirq.GridQubit(1, 0)
q11 = cirq.GridQubit(1, 1)
q12 = cirq.GridQubit(1, 2)
q20 = cirq.GridQubit(2, 0)
q21 = cirq.GridQubit(2, 1)
q22 = cirq.GridQubit(2, 2)
c = cirq.Circuit.from_ops(
    #cirq.CNOT(q00, q01), cirq.Z(q01) ** 0.1, cirq.CNOT(q00, q01),

    cirq.SWAP(q00, q10),
    cirq.SWAP(q10, q20),
    cirq.SWAP(q20, q21),
    cirq.SWAP(q21, q22),
    cirq.SWAP(q22, q12),
    cirq.SWAP(q12, q02),
    cirq.CNOT(q02, q01), cirq.Z(q01)**0.2, cirq.CNOT(q02, q01),
    cirq.SWAP(q12, q02),
    cirq.SWAP(q22, q12),
    cirq.SWAP(q21, q22),
    cirq.SWAP(q20, q21),
    cirq.SWAP(q10, q20),
    cirq.SWAP(q00, q10),
    
    cirq.CNOT(q00, q01), cirq.Z(q01)**0.1, cirq.CNOT(q00, q01),
)
s = c.to_text_diagram(qubit_order=(q00, q10, q20, q21, q22, q12, q02,
                               q01))
print(s)

(0, 0): ───×───────────────────────────────────────────────────────────×───@───────────@───
           │                                                           │   │           │
(1, 0): ───×───×───────────────────────────────────────────────────×───×───┼───────────┼───
               │                                                   │       │           │
(2, 0): ───────×───×───────────────────────────────────────────×───×───────┼───────────┼───
                   │                                           │           │           │
(2, 1): ───────────×───×───────────────────────────────────×───×───────────┼───────────┼───
                       │                                   │               │           │
(2, 2): ───────────────×───×───────────────────────────×───×───────────────┼───────────┼───
                           │                           │                   │           │
(1, 2): ───────────────────×───×───────────────────×───×───────────────────┼───────────┼───
   

In [16]:
cc = paulistring.optimized_circuit(c)
cc

(0, 0): ───X^0.5───@───[Y]^-0.3───@───X^-0.5───
                   │              │
(0, 1): ───────────@──────────────@────────────

In [17]:
d = cc.to_text_diagram_drawer(
        qubit_order=(q00, q10, q20, q21, q22, q12, q02, q01))
s = d.render(horizontal_spacing=0)
print(s)

(0, 0)X^0.5@[Y]^-0.3@X^-0.5
           │        │
(1, 0)─────┼────────┼──────
           │        │
(2, 0)─────┼────────┼──────
           │        │
(2, 1)─────┼────────┼──────
           │        │
(2, 2)─────┼────────┼──────
           │        │
(1, 2)─────┼────────┼──────
           │        │
(0, 2)─────┼────────┼──────
           │        │
(0, 1)─────@────────@──────


In [18]:
q0, q1, q2 = cirq.LineQubit.range(3)
c = cirq.Circuit.from_ops(
    cirq.T(q2),
    cirq.CNOT(q1, q2),
    cirq.CNOT(q2, q1),
    cirq.CNOT(q0, q1),
    cirq.CNOT(q1, q0),
    cirq.T(q0),
)
display(c)
cirq.ExpandComposite().optimize_circuit(c)
c = cirq.Circuit.from_ops(c.all_operations(), strategy=cirq.InsertStrategy.EARLIEST)
display(c)
c = paulistring.converted_gate_set(c)
c

0: ───────────────@───X───T───
                  │   │
1: ───────@───X───X───@───────
          │   │
2: ───T───X───@───────────────

0: ──────────────────────────────────────────────────@───Y^-0.5───@───Y^0.5───T───
                                                     │            │
1: ────────────────@───Y^-0.5───@───Y^0.5───Y^-0.5───@───Y^0.5────@───────────────
                   │            │
2: ───T───Y^-0.5───@───Y^0.5────@─────────────────────────────────────────────────

0: ────────────────────────────────────────────────@───Y^-0.5───@───Y^0.5───[Z]^0.25───
                                                   │            │
1: ───────────────────────────────@───Y^-0.5───@───@───Y^0.5────@──────────────────────
                                  │            │
2: ───[Z]^0.75───(X^0.5-Z^-0.5)───@───Y^0.5────@───────────────────────────────────────

In [19]:
paulistring.optimized_circuit(c)

0: ────────────────────────────@────────Y^-0.5───@───(Z-Y^-0.5)───
                               │                 │
1: ───────────@───Y^-0.5───@───@────────Y^0.5────@────────────────
              │            │
2: ───X^0.5───@───X^-0.5───@───Z^-0.5─────────────────────────────

In [20]:
cirq.two_qubit_matrix_to_operations(
q0, q1, cirq.CNOT.matrix(), allow_partial_czs=False)

[GateOperation(Z**0.75, (LineQubit(0),)),
 GateOperation(X**0.5, (LineQubit(0),)),
 GateOperation(S**-1, (LineQubit(0),)),
 GateOperation(X**-0.25, (LineQubit(1),)),
 GateOperation(Y**-0.5, (LineQubit(0),)),
 GateOperation(Y**-0.5, (LineQubit(1),)),
 GateOperation(CZ, (LineQubit(0), LineQubit(1))),
 GateOperation(S**-1, (LineQubit(0),)),
 GateOperation(S**-1, (LineQubit(1),)),
 GateOperation(Y**0.5, (LineQubit(0),)),
 GateOperation(Y**0.5, (LineQubit(1),)),
 GateOperation(Y**0.5, (LineQubit(0),)),
 GateOperation(Z**-0.75, (LineQubit(0),)),
 GateOperation(X**-0.25, (LineQubit(1),))]

In [21]:
cirq.X(q0) ** .5

GateOperation(X**0.5, (LineQubit(0),))

In [22]:
(cirq.X**0.5).on(cirq.NamedQubit('q0'))

GateOperation(X**0.5, (NamedQubit('q0'),))

In [23]:
c = cirq.Circuit.from_ops(
    cirq.X(q0) ** .1,
    cirq.Y(q0) ** .2,
    cirq.Z(q0) ** .3,
    cirq.H(q0),
    cirq.google.ExpWGate(axis_half_turns=.4, half_turns=.5)(q0),
    cirq.X(q0) ** .6,
    cirq.Y(q0) ** .7,
    cirq.Z(q0) ** .8,
)
c

0: ───X^0.1───Y^0.2───Z^0.3───H───W(0.4)^0.5───X^0.6───Y^0.7───Z^0.8───

In [24]:
cc = paulistring.optimized_circuit(c)
cc

0: ───[Z]^0.154───[X]^0.659───[Z]^-0.101───

In [25]:
cirq.google.optimized_for_xmon(cc)

0: ───W(0.846)^-0.659───Z^0.0531───

In [26]:
c = cirq.Circuit.from_ops(
    cirq.google.ExpWGate(axis_half_turns=.1, half_turns=.2)(q0),
    cirq.google.ExpWGate(axis_half_turns=.3, half_turns=.4)(q1),
    cirq.CZ(q0, q1),
    cirq.H(q0),
    cirq.X(q1) ** .5,
    cirq.ISWAP(q0, q1),
    cirq.google.ExpWGate(axis_half_turns=.6, half_turns=.7)(q0),
    cirq.google.ExpWGate(axis_half_turns=.8, half_turns=.9)(q1),
    cirq.CNOT(q0, q1),
    cirq.X(q0) ** .1,
    cirq.Y(q1) ** .2,
    cirq.CNOT(q0, q1),
    cirq.Z(q0) ** .3,
    cirq.X(q1) ** .4,
)
c

0: ───W(0.1)^0.2───@───H───────iSwap───W(0.6)^0.7───@───X^0.1───@───Z^0.3───
                   │           │                    │           │
1: ───W(0.3)^0.4───@───X^0.5───iSwap───W(0.8)^0.9───X───Y^0.2───X───X^0.4───

In [27]:
cirq.google.optimized_for_xmon(c)

0: ───W(0.0037)^0.783───@───W(0.389)^0.5──────@───W(0.311)^-0.5─────@───W(0.129)^0.534────Z^-0.597───
                        │                     │                     │
1: ───W(0.696)^0.233────@───W(0.647)^-0.867───@───W(0.647)^-0.983───@───W(0.709)^-0.416───Z^0.732────

In [28]:
c = cirq.Circuit.from_ops(
    cirq.google.ExpWGate(axis_half_turns=.1, half_turns=.2)(q0),
    cirq.google.ExpWGate(axis_half_turns=.3, half_turns=.4)(q1),
    cirq.CZ(q0, q1),
    cirq.H(q0),
    cirq.X(q1) ** .5,
    cirq.ISWAP(q0, q1),
    cirq.google.ExpWGate(axis_half_turns=.6, half_turns=.7)(q0),
    cirq.google.ExpWGate(axis_half_turns=.8, half_turns=.9)(q1),
    cirq.CNOT(q0, q1),
    cirq.X(q0) ** .1,
    cirq.Y(q1) ** .2,
    cirq.CNOT(q0, q1),
    cirq.Z(q0) ** .3,
    cirq.X(q1) ** .4,
)
c

0: ───W(0.1)^0.2───@───H───────iSwap───W(0.6)^0.7───@───X^0.1───@───Z^0.3───
                   │           │                    │           │
1: ───W(0.3)^0.4───@───X^0.5───iSwap───W(0.8)^0.9───X───Y^0.2───X───X^0.4───

In [29]:
c = cirq.Circuit.from_ops(
    cirq.CNOT(q0, q1)**.5,
)
c

0: ───@───────
      │
1: ───X^0.5───

In [30]:
paulistring.optimized_circuit(c)

0: ───X^0.5───@───[Y]^-0.25───@───[Y]^-0.25───X^-0.5───────────
              │               │
1: ───Y^0.5───@───────────────@───[Z]^0.25────(X^0.5-Z^-0.5)───

In [31]:
cirq.google.optimized_for_xmon(c)

0: ───W(0.357)^0.5───@───W(0.857)^-0.25───@───W(0.357)^-0.5───Z^0.25────
                     │                    │
1: ───Y^0.5──────────@────────────────────@───W(0.75)^-0.5────Z^-0.25───

In [32]:
ops = cirq.ops
c = cirq.Circuit.from_ops(
    ops.Y(q2) ** 0.5,
    ops.X(q2),
    ops.CNOT(q1, q2),
    ops.Z(q2) ** -0.25,
    ops.SWAP(q1, q2),
    ops.CNOT(q0, q1),
    ops.SWAP(q1, q2),
    ops.Z(q2) ** 0.25,
    ops.CNOT(q1, q2),
    ops.Z(q2) ** -0.25,
    ops.SWAP(q1, q2),
    ops.CNOT(q0, q1),
    ops.SWAP(q1, q2),
    ops.Z(q2) ** 0.25,
    ops.Z(q1) ** 0.25,
    ops.CNOT(q0, q1),
    ops.Z(q0) ** 0.25,
    ops.Z(q1) ** -0.25,
    ops.CNOT(q0, q1),
    ops.Y(q2) ** 0.5,
    ops.X(q2),
)
c

0: ──────────────────────────────@──────────────────────────@───────────@───T──────@───────────
                                 │                          │           │          │
1: ───────────────@──────────×───X───×───────@──────────×───X───×───T───X───T^-1───X───────────
                  │          │       │       │          │       │
2: ───Y^0.5───X───X───T^-1───×───────×───T───X───T^-1───×───────×───T──────────────Y^0.5───X───

In [33]:
d = c.to_text_diagram_drawer()
s = d.render(horizontal_spacing=1)
print(s)

0──────────────────@──────────────@─────@─T────@───────
                   │              │     │      │
1─────────@──────×─X─×───@──────×─X─×─T─X─T^-1─X───────
          │      │   │   │      │   │
2─Y^0.5─X─X─T^-1─×───×─T─X─T^-1─×───×─T────────Y^0.5─X─
